## Experiment 3
Apply a-priori algorithm to find frequently occurring items from given data and generate strong association rules using support and confidence thresholds. For Example: Market Basket Analysis .

Performed on kaggle.com
Dataset available on https://www.kaggle.com/mashlyn/online-retail-ii-uci

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import mlxtend

In [2]:
df = pd.read_csv('online_retail_II.csv').dropna()
df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom


In [3]:
items = df[['StockCode','Description']].drop_duplicates()
items.head()

,StockCode,Description
0,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS
1,79323P,PINK CHERRY LIGHTS
2,79323W,WHITE CHERRY LIGHTS
3,22041,"RECORD FRAME 7"" SINGLE SIZE"
4,21232,STRAWBERRY CERAMIC TRINKET BOX


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4033 entries, 0 to 4952
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Invoice      4033 non-null   object 
 1   StockCode    4033 non-null   object 
 2   Description  4033 non-null   object 
 3   Quantity     4033 non-null   int64  
 4   InvoiceDate  4033 non-null   object 
 5   Price        4033 non-null   float64
 6   Customer ID  4033 non-null   float64
 7   Country      4033 non-null   object 
dtypes: float64(2), int64(1), object(5)
memory usage: 283.6+ KB


# **Creating Dataset**
We group items having the same invoice number (i.e. they have been purchased together) into lists

In [5]:
dataset = df.groupby(['Invoice'])['StockCode'].apply(list)
dataset.head()

'''
for reference: 
https://stackoverflow.com/questions/53037888/pandas-groupby-to-list
'''

'\nfor reference: \nhttps://stackoverflow.com/questions/53037888/pandas-groupby-to-list\n'

In [6]:
#converting to list
dataset = dataset.tolist()[:5000] #using only first 3000 entries for this experiment
print(dataset[:5])

[['85048', '79323P', '79323W', '22041', '21232', '22064', '21871', '21523'], ['22350', '22349', '22195', '22353'], ['48173C', '21755', '21754', '84879', '22119', '22142', '22296', '22295', '22109', '22107', '22194', '35004B', '82582', '21181', '21756', '21333', '84596F', '84596L', '22111'], ['22143', '22145', '22130', '21364', '21360', '21351', '21352', '35400', '20695', '37370', '10002', '84507B', '20703', '21987', '21989', '84970S', '20971', '22271', '22272', '22274', '21912', '22111', '22112'], ['21329', '21252', '21100', '21033', '20711', '21410', '21411', '84031A', '84031B', '84032A', '84032B', '84519A', '84519B', '85132A', '85132C', '85183A', '85183B']]


# Apriori Implementation

In [7]:
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori 
from mlxtend.frequent_patterns import association_rules

In [8]:
# encoding transactions
te = TransactionEncoder()
encoded_dataset = te.fit(dataset).transform(dataset)
transaction_df = pd.DataFrame(encoded_dataset,columns=te.columns_)
transaction_df.head()

,10002,10120,10125,10133,10135,10138,11001,15034,15036,15044B,15044D,15056BL,15056N,15056P,16012,16047,16156N,16156S,16161C,16161P,16161U,16169C,16169D,16169F,16169N,16207A,16212,16225,16235,16237,16238,16258B,17003,17033,17084J,17084R,17091J,17108D,17109A,17109B,...,90092,90093,90121B,90125A,90125C,90125D,90125E,90129B,90131,90134,90139,90145,90147,90148,90155,90167,90168,90175A,90175D,90180A,90185B,90185C,90190A,90195B,90199C,90200A,90200B,90200C,90200D,90200E,90203,90209A,90209C,90214A,90214H,90214J,90214S,D,M,POST
0,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
3,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False


In [9]:
#obtaining frequent itemsets
frequent_itemsets=apriori(transaction_df,min_support=0.01,use_colnames=True) # lower min support for large dataset
frequent_itemsets.count()

support     2045
itemsets    2045
dtype: int64

In [10]:
frequent_itemsets

,support,itemsets
0,0.013043,(10135)
1,0.021739,(15056BL)
2,0.021739,(15056N)
3,0.013043,(16161U)
4,0.017391,(16169D)
...,...,...
2040,0.013043,"(90084, 90087, 90092, 90093, 90090, 90085)"
2041,0.013043,"(90084, 90088, 90092, 90093, 90090, 90085)"
2042,0.013043,"(90084, 90088, 90092, 90087, 90093, 90090)"
2043,0.013043,"(90088, 90092, 90087, 90093, 90090, 90085)"


In [11]:
res=association_rules(frequent_itemsets,metric='confidence',min_threshold=0.005)
res

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(21875),(10135),0.021739,0.013043,0.013043,0.6000,46.000000,0.012760,2.467391
1,(10135),(21875),0.013043,0.021739,0.013043,1.0000,46.000000,0.012760,inf
2,(22125),(10135),0.069565,0.013043,0.013043,0.1875,14.375000,0.012136,1.214716
3,(10135),(22125),0.013043,0.069565,0.013043,1.0000,14.375000,0.012136,inf
4,(15056N),(15056BL),0.021739,0.021739,0.013043,0.6000,27.600000,0.012571,2.445652
...,...,...,...,...,...,...,...,...,...
8099,(90087),"(90084, 90088, 90092, 90093, 90090, 90085)",0.013043,0.013043,0.013043,1.0000,76.666667,0.012873,inf
8100,(90092),"(90084, 90088, 90087, 90093, 90090, 90085)",0.013043,0.013043,0.013043,1.0000,76.666667,0.012873,inf
8101,(90093),"(90084, 90088, 90092, 90087, 90090, 90085)",0.013043,0.013043,0.013043,1.0000,76.666667,0.012873,inf
8102,(90090),"(90084, 90088, 90092, 90087, 90093, 90085)",0.013043,0.013043,0.013043,1.0000,76.666667,0.012873,inf


In [12]:
#function to obtain item description for given stock code
def re_label(codes):
    labels = []
    for code in codes:
        labels.append(items[items['StockCode']==code]['Description'].iloc[0])
    return tuple(labels)

In [13]:
res2 = res[res['confidence']>=1]
res2 = res2.reset_index()
res2.head()

,index,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,1,(10135),(21875),0.013043,0.021739,0.013043,1.0,46.000000,0.012760,inf
1,3,(10135),(22125),0.013043,0.069565,0.013043,1.0,14.375000,0.012136,inf
2,7,(17003),(85099B),0.013043,0.082609,0.013043,1.0,12.105263,0.011966,inf
3,21,(21584),(20675),0.013043,0.034783,0.013043,1.0,28.750000,0.012590,inf
4,54,(20681),(85099B),0.017391,0.082609,0.017391,1.0,12.105263,0.015955,inf


In [14]:
#sample row index 43354
print('antecedents:')
print(re_label(list(res2.iloc[3616]['antecedents'])))
print('consequents:')
print(re_label(list(res2.iloc[3616]['consequents'])))

antecedents:
('CRYSTAL SEA HORSE PHONE CHARM',)
consequents:
('PINK CRYSTAL GUITAR PHONE CHARM', 'CRYSTAL KEY+LOCK PHONE CHARM', 'BLUE CRYSTAL BOOT PHONE CHARM', 'CLEAR CRYSTAL STAR PHONE CHARM', 'PINK CRYSTAL HEART PHONE CHARM', 'CRYSTAL STILETTO PHONE CHARM')


In [15]:
#sample row index 0
print('antecedents:')
print(re_label(list(res2.iloc[0]['antecedents'])))
print('consequents:')
print(re_label(list(res2.iloc[0]['consequents'])))

antecedents:
('COLOURING PENCILS BROWN TUBE',)
consequents:
('KINGS CHOICE MUG',)
